# 04 - The Hydrogen Molecular Ion $H_2^+$

**Overview** 

This notebook guides you through the calculation of the electronic states of the H2+ molecule as a function of the separation between the hydrogen atoms. As the two atoms get further apart, the solutions of the quantum mechanical problem will converge to the isolated hydrogen atom levels. However, as the atoms come close, we want to understand if the only electron in the system will be able to stabilize the molecule and create a chemical bonded molecule.

In [ ]:
# @title Modules Setup { display-mode: "form" }
import numpy as np
# Install Plotly (if not already)
!pip install -q plotly > /dev/null
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import matplotlib as mpl
!pip install -q rdkit > /dev/null
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from scipy.special import genlaguerre, sph_harm, factorial
from numpy.polynomial.legendre import leggauss
from numpy.polynomial.laguerre import laggauss
from scipy.linalg import eigh
from functools import partial

In [ ]:
# @title Utility Functions  { display-mode: "form" }
# -------------------- Basis Set Generation --------------------
def sph_coords(rvec):
    """
    Convert Cartesian coordinates to spherical coordinates.
    """
    x, y, z = rvec[...,0], rvec[...,1], rvec[...,2]
    r = np.sqrt(x*x + y*y + z*z)
    # theta: handle r=0 safely
    theta = np.where(r > 0, np.arccos(np.clip(z / (r + 1e-300), -1.0, 1.0)), 0.0)
    phi = np.arctan2(y, x)
    return r, theta, phi

def R_nl(n, l, r, Z=1):
    """
    Hydrogenic radial wavefunction (atomic units, a0=1).
    Returns R_{n,l}(r).
    """
    # Normalization constant
    prefac = np.sqrt((2.0*Z/n)**3 * factorial(n-l-1) / (2*n*factorial(n+l)))
    rho = 2.0 * Z * r / n
    L = genlaguerre(n-l-1, 2*l+1)(rho)  # Associated Laguerre polynomial

    return prefac * np.exp(-rho/2) * rho**l * L

def hydrogen_orbital(n, l, m, r0, rvec, Z=1):
    """
    Full hydrogenic orbital psi_{n,l,m}(rvec-r0).
    - rvec: cartesian coordinate (array)
    - r0: center of orbital (array)
    """
    r, theta, phi = sph_coords(rvec - r0)
    R = R_nl(n, l, r, Z)
    Y = sph_harm(m, l, phi, theta)  # scipy uses (m,l,phi,theta)
    return R * Y

def hydrogen_energy(n, Z=1):
    """
    Energy of hydrogen-like orbital with principal quantum number n,
    nuclear charge Z (default hydrogen), in Hartree atomic units.

    E_n = - Z^2 / (2 n^2)
    """
    if n < 1 or int(n) != n:
        raise ValueError("n must be a positive integer")
    return - (Z**2) / (2.0 * n**2)

# -------------------- Partition of unity weights --------------------
def weights_RA_RB(r, RA, RB, alpha=3.0):
    dA = np.linalg.norm(r - RA)
    dB = np.linalg.norm(r - RB)
    R  = np.linalg.norm(RB - RA) + 1e-300
    z  = 2.0 * alpha * (dB - dA) / R
    wA = 1.0 / (1.0 + np.exp(-z))   # ~1 near A, ~0 near B
    return wA, 1.0 - wA

# -------------------- Atom-centered spherical quadrature --------------------
def integrate_atom_centered(f, R0, n_rad=40, n_th=20, n_ph=24, alpha_rad=1.0):
    """
    ∫ f(r) d^3r using spherical coords about R0.
    Radial: Gauss–Laguerre with mapping r = alpha_rad * x (weight e^{-x})
    Angular: Gauss–Legendre in cosθ; φ uniform
    """
    # Radial Gauss–Laguerre for ∫_0^∞ e^{-x} g(x) dx
    x_r, w_r = laggauss(n_rad)
    r = alpha_rad * x_r
    # Angular nodes/weights
    u, w_th = leggauss(n_th)           # u = cosθ in [-1,1]
    th = np.arccos(u)
    ph = np.linspace(0.0, 2.0*np.pi, n_ph, endpoint=False)
    w_ph = (2.0*np.pi)/n_ph * np.ones_like(ph)

    total = 0.0
    for ri, wri in zip(r, w_r):
        # Jacobian: (alpha^3) * r^2 e^{-r/alpha} * sinθ
        pref_r = alpha_rad * wri * np.exp(ri/alpha_rad) * (ri**2)
        sin_th = np.sin(th)
        for thi, wti, sin_i in zip(th, w_th, sin_th):
            for phi, wpi in zip(ph, w_ph):
                x_i = R0[0] + ri*np.sin(thi)*np.cos(phi)
                y_i = R0[1] + ri*np.sin(thi)*np.sin(phi)
                z_i = R0[2] + ri*np.cos(thi)
                jac = pref_r * wti * wpi
                total += f(np.array([x_i, y_i, z_i])) * jac
    return total

# -------------------- Overlap S integral for orbitals i,j ----------------------------------
def S_i_j(orbital_i, orbital_j, RA, RB, n_rad=40, n_th=20, n_ph=24, alpha=1.0):
    """
    S_i_j = ∫ φ_{j}(r)* x φ_{i}(r) d^3r
    Evaluated with two-grid partition-of-unity by default.
    """

    def raw_integrand(r):
        val = orbital_i(rvec = r)*np.conj(orbital_j(rvec = r))
        return val

    # Two-grid split with smooth weights
    def fA(r):
        wA, _ = weights_RA_RB(r, RA, RB)
        return wA * raw_integrand(r)

    def fB(r):
        _, wB = weights_RA_RB(r, RA, RB)
        return wB * raw_integrand(r)

    IA = integrate_atom_centered(fA, RA, n_rad, n_th, n_ph, alpha_rad=alpha)
    IB = integrate_atom_centered(fB, RB, n_rad, n_th, n_ph, alpha_rad=alpha)
    return IA + IB

# ----------- Coulomb J integral for orbitals i and j on center opposite to i --------------
def J_i_j(orbital_i, orbital_j, RA, RB, n_rad=40, n_th=20, n_ph=24, alpha=1.0):
    """
    J_iA^(B)(R) = -∫ φ_{j}(r)* x φ_{i}(r) / |r - RB| d^3r
    Evaluated with two-grid partition-of-unity by default.
    """

    def raw_integrand(r):
        # Compute Coulomb integral with nucleus opposite to i-th orbital center
        R0 = -orbital_i.keywords.get("r0")
        val = orbital_i(rvec = r)*np.conj(orbital_j(rvec = r))
        return - val / (np.linalg.norm(r - R0) + 1e-300)

    # Two-grid split with smooth weights
    def fA(r):
        wA, _ = weights_RA_RB(r, RA, RB)
        return wA * raw_integrand(r)

    def fB(r):
        _, wB = weights_RA_RB(r, RA, RB)
        return wB * raw_integrand(r)

    IA = integrate_atom_centered(fA, RA, n_rad, n_th, n_ph, alpha_rad=alpha)
    IB = integrate_atom_centered(fB, RB, n_rad, n_th, n_ph, alpha_rad=alpha)
    return IA + IB

# Build Overlap Matrix
def overlap_matrix(basis, RA, RB, n_rad=40, n_th=20, n_ph=24, alpha=1.0):

    S = np.zeros((len(basis),len(basis)), dtype=np.complex128)

    for i, ket in enumerate(basis):
        for j, bra in enumerate(basis):
            if i == j:
                # the basis is normalized
                S[i,j] = 1.0
            elif i < j :
                # if the two basis have the same center, they should be orthogonal
                if np.array_equal(ket.keywords.get("r0"), bra.keywords.get("r0")):
                    S[i,j] = 0.0
                else:
                    # otherwise, compute the overlap
                    S[i,j] = S_i_j(ket, bra, RA, RB, n_rad, n_th, n_ph, alpha)
                    S[j,i] = np.conj(S[i,j])
    return S

# Build Hamiltonian Matrix
def hamiltonian_matrix(basis_vectors, basis_energies, RA, RB, n_rad=40, n_th=20, n_ph=24, alpha=1.0):

    H = np.zeros((len(basis_vectors),len(basis_vectors)), dtype=np.complex128)
    S = overlap_matrix(basis_vectors, RA, RB, n_rad, n_th, n_ph, alpha)

    # we can exploit symmetry to compute only the two top halves of the top quarters of the matrix
    for i, (ket, ei) in enumerate(zip(basis_vectors[::2],basis_energies[::2])):
        i_index = i*2 # actual index in the full basis
        for j, bra in enumerate(basis_vectors[i_index::2]):
            j_index = i_index + 2*j
            H[i_index,j_index] = ei*S[i_index,j_index] + J_i_j(ket, bra, RA, RB, n_rad, n_th, n_ph, alpha)
            H[j_index,i_index] = np.conj(H[i_index,j_index])
            H[i_index+1,j_index+1] = H[i_index,j_index]
            H[j_index+1,i_index+1] = np.conj(H[i_index,j_index])
        for j, bra in enumerate(basis_vectors[i_index+1::2]):
            j_index = i_index + 1 + 2*j
            H[i_index,j_index] = ei*S[i_index,j_index] + J_i_j(ket, bra, RA, RB, n_rad, n_th, n_ph, alpha)
            H[j_index,i_index] = np.conj(H[i_index,j_index])
            H[i_index+1,j_index-1] = H[i_index,j_index]
            H[j_index-1,i_index+1] = np.conj(H[i_index,j_index])
    return H, S

# -------------------- Generalized Eigenvalue Problem Solver --------------------
def generalized_eigh_lowdin(H, S, rel_cut=5e-2):
    # S = U diag(w) U^H
    w, U = eigh(S)
    thr = rel_cut * w.max()
    keep = w > thr
    if not np.any(keep):
        raise np.linalg.LinAlgError("All overlap eigenvalues below threshold.")
    Uk, wk = U[:, keep], w[keep]

    S_inv_sqrt = (Uk / np.sqrt(wk)) @ Uk.conj().T
    H_tilde    = S_inv_sqrt @ H @ S_inv_sqrt
    E, Ctil    = eigh(H_tilde)
    C          = S_inv_sqrt @ Ctil           # AO-space eigenvectors; C^H S C ≈ I
    return E, C

def metric_gram_schmidt(S, rel_tol=1e-10, abs_tol=0.0):
    """
    Order-preserving S-metric Gram–Schmidt on the canonical basis {e_i}.
    Returns Q (n×k) with Q^H S Q = I, and the kept AO indices.

    rel_tol: drop if ||res||_S <= rel_tol * sqrt(S[ii])
    abs_tol: or if ||res||_S <= abs_tol (absolute safety floor)
    """
    S = np.asarray(S)
    n = S.shape[0]
    Q = np.zeros((n, 0), dtype=complex)
    keep = []

    # precompute sqrt of diagonal as scaling
    sdiag = np.sqrt(np.clip(np.real(np.diag(S)), 0.0, np.inf))

    for i in range(n):
        # residual in coefficient space: v = e_i - Q (Q^H S e_i)
        s_col = S[:, i]
        alpha = Q.conj().T @ s_col           # projections in S-metric
        v = np.zeros(n, dtype=complex); v[i] = 1.0
        if Q.shape[1] > 0:
            v = v - Q @ alpha

        # S-norm of residual: ||v||_S^2 = v^H S v = S[ii] - alpha^H alpha
        # (use the formula for stability; v^H S v works too)
        r2 = float(max(sdiag[i]**2 - np.vdot(alpha, alpha).real, 0.0))
        r = np.sqrt(r2)

        # drop criterion (relative to sqrt(S[ii]) plus absolute floor)
        if r <= max(rel_tol * (sdiag[i] if sdiag[i] > 0 else 1.0), abs_tol):
            continue  # nearly dependent → skip
        q = v / r
        Q = np.column_stack((Q, q))
        keep.append(i)

    return Q, np.array(keep, dtype=int)

def solve_with_mgs(H, S, rel_tol=1e-10, abs_tol=0.0):
    """
    Solve generalized problem Hc = E S c via MGS (order-preserving).
    """
    Q, keep = metric_gram_schmidt(S, rel_tol=rel_tol, abs_tol=abs_tol)
    Ht = Q.conj().T @ H @ Q
    E, Y = eigh(Ht)                  # ordinary Hermitian EVP
    C = Q @ Y                        # AO coefficients; C^H S C ≈ I
    return E, C

def ghost_score(S, c, lam_cut=1e-2, relative=True):
    # S = U diag(λ) U^H
    lam, U = eigh(S)
    if relative:
        lam_cut = lam_cut * lam.max()
    y = U.conj().T @ c
    small = lam < lam_cut
    # fraction of Euclidean weight living in small-λ subspace
    num = np.sum(np.abs(y[small])**2)
    den = np.sum(np.abs(y)**2)
    return float(num/den), lam, U
# Example usage:
#scores = [ghost_score(S, C[:,k])[0] for k in range(C.shape[1])]
#print(scores)
# A ghost will have score ~ 1.0

**Problem** 

We will consider the H2+ molecule, composed by two hydrogen nuclei and one electron. For convenience, we will orient the molecule along the x axis and center it on the origin. We want to study the total energy of the system as the distance between the two atoms is varied, so as to understand if there is a stable bond in the molecule. In order to do that, we first need to make sure our calculations are accurate enough and reproduce the asymptotic behavior of the system.

**Model**
Our underlying model is represented by the quantum-mechanical treatment of the single electron in a potential given by the two nuclei, considered as fixed. This is the so-called Born-Oppeneimer approximation and it is based on the huge difference in mass between the electron and the protons. The Hamiltonian of the system in the coordinates representation is thus

$$
\left[-\frac{\hbar^2}{2m}\nabla^2 - \frac{1}{\left|\vec{r}-\vec{R}_A\right|}- \frac{1}{\left|\vec{r}-\vec{R}_B\right|} \right]\psi(\vec{r})=E\psi(\vec{r})
$$

where $\vec{R}_A$ and $\vec{R}_B$ are the positions of the nuclei, $\vec{r}$ are the coordinates of the electron, $E$ and $\psi$ are the energies and wavefunctions that solve the time-independent Schrodinger equation. 

Instead of trying to solve the above equation in the coordinate representation (e.g. by discretizing the Cartesian space and adjusting the values of the function at each point until the equation is solved), we are going to rely on a smarter basis. We recognize that the above Hamiltonian is just the one of hydrogen atom for atom A plus the coulomb interaction with nucleus B (or vice-versa, with A and B swapped):

$$
\hat{H} = \hat{H}^0_A + \hat{V}_B = \hat{H}^0_B + \hat{V}_A 
$$

Since we know the eigenvalues and eigenvectors of the hydrogen atom Hamiltonian, we can solve exactly and with minimal effort the H2+ problem if we express it on a basis composed by hydrogenoid functions (1s, 2s, 2p, etc.) centered on each of the two atoms. We can thus express our sought state as a linear combination of this (in principle infinite) basis:

$$
\psi = c_{1sA} \phi_{1sA} + c_{1sB} \phi_{1sB} + c_{2sA} \phi_{2sA} + c_{2sB} \phi_{2sB} + \dots = \left[
\begin{array}{c}  % 'c' = centered; use 'l' or 'r' if you prefer
c_{1sA} \\ c_{1sB} \\ c_{2sA} \\c_{2sB} \\ \dots
\end{array}
\right] \equiv \mathbf{c}
$$

Contrary to all the examples considered in class so far, this basis is normalized, but it is not fully ortogonal: the functions centered on atom A are not ortogonal to those centered on atom B, they have a non-zero overlap. In this situation, the equation to solve is a generalized eigenvalue problem

$$
\mathbf{H}\mathbf{c} = E\mathbf{c}\mathbf{S}
$$

where the elements of the Hamiltonian matrix are 

$$
H_{i,j} = \left<\phi_i|\hat{H}|\phi_j\right> = \epsilon^0_i +  \left<\phi_i|\hat{V}_X|\phi_j\right> 
$$

while the overlap matrix S has elements 

$$
S_{i,j} = \left<\phi_i|\phi_j\right>
$$

In order to build the two matrices we will need to evaluate overlap and coulomb integrals that involve the states of the hydrogen atoms. We will do it numerically, with a procedure (quadrature) that can be systematically improved. We thus have, in principles, acces to the exact solutions of the QM problem of the H2+ molecule.

>How many electronic states can we describe and how many of these are compatible with a bound molecule?


**Questions**

Before you run any simulation, answer the following question(s):

1. Knowing the formula for the energy levels of the hydrogen atom ($\epsilon^0_n = 1/2n^2$ in a.u.) and that the nucleus-nucleus repulsion is $1/|RA-RB|$ in a.u., what is the energy of the system when the two protons are infinitely apart? 
2. What is the energy as the distance between the two protons goes to zero?
3. Do you think that the total energy of the system as a function of the distance of the two protons has a minimum? In case, can you guess the distance at which the minimum occurs? 

The calculations rely on two ortogonal numerical paramters: the number of functions in our basis set ($N^{basis}$) and the number of integration points used to compute the integrals and build the $\mathbf{H}$ and $\mathbf{S}$ matrices. Ideally both parameters would be infinite, using a finite number corresponds to introducing a truncation error.

Run the first part of the notebook, change the parameters, and run the calculations again as many times as needed to answer the following question(s):
4. Consider a minimal basis (only the 1s functions) and check how the results change as you improve the accuracy of the integration. How does the cost scale? 
5. Now consider a coarse integration grid and check how the results change as you increase the basis. 
6. Can you derive an expression on how the cost of setting up the matrices depend on the size of the basis and on the size of the integration grid? Which numerical parameter affects the scaling the most? 
7. For a given choice of the numerical parameters, verify if the solutions obtained have the correct asymptotic values.
8. What is the shape of the lowest two states? Could you have predicted this result without running any calculation?

Run the last part of the notebook to compute the potential energy surface of the different electronic states of the H2+ molecule. 
9. For how many of the states the molecule is bound? 
10. At what distance is the total energy of the system at its minimum? 

In [ ]:
# @title Simulation Parameters  { display-mode: "form" }
distance = 3.0 # @param {type:"number"}
nmax = 1  # @param {type:"integer"}
lmax = 0  # @param {type:"integer"}
integration_accuracy = "coarse"  # @param ["coarse", "medium", "fine", "very fine"]
solver = 'mgs'  # @param ["mgs", "lowdin"]
#logical = True # @param {type:"boolean"} --- IGNORE ---

In [ ]:
# @title Setup and Solve the Generalized Eigenvalue Problem { display-mode: "form" }

RA = np.array([-distance/2, 0.0, 0.0])  # nucleus A at -R/2 on x-axis
RB = np.array([ distance/2, 0.0, 0.0])  # nucleus B at +R/2 on x-axis

ns = []
orbitals = []
for i in range(1,nmax+1):
    for j in range(0, min(lmax+1,i)):
        for m in range(-j, j+1):
            ns.append(i)
            orbitals.append(partial(hydrogen_orbital, n=i, l=j, m=m))

basis_vectors = []
basis_energies = []
for n, orbital in zip(ns, orbitals):
    for atom_position in [RA, RB]:
        basis_vectors.append(partial(orbital, r0=atom_position))
        basis_energies.append(hydrogen_energy(n))

if integration_accuracy == "coarse":
    n_rad, n_th, n_ph = 10, 10, 12
elif integration_accuracy == "medium":
    n_rad, n_th, n_ph = 20, 20, 24
elif integration_accuracy == "fine":
    n_rad, n_th, n_ph = 40, 30, 24
elif integration_accuracy == "very fine":
    n_rad, n_th, n_ph = 50, 40, 36

print(f"Using n_rad={n_rad}, n_th={n_th}, n_ph={n_ph} for spherical quadrature.")
print(f"Total number of integration points: {2 * n_rad * n_th * n_ph}")
print(f"Total number of basis functions: {len(basis_vectors)}")

H, S = hamiltonian_matrix(basis_vectors, basis_energies, RA, RB, n_rad, n_th, n_ph, alpha=1.0)
if solver == 'mgs':
    E, C = solve_with_mgs(H, S, rel_tol=1e-10, abs_tol=0.0)
elif solver == 'lowdin':
    E, C = generalized_eigh_lowdin(H, S, rel_cut=5e-2)

In [ ]:
# @title Visualize the Wavefunctions { display-mode: "form" }
type = "eigenvector"  # @param ["basis", "eigenvector"]
show = "wavefunction" # @param ["wavefunction", "density"]
index = 0 # @param {type:"integer"}
if index < 0 or index >= len(basis_vectors):
    raise ValueError(f"Index out of range. Must be between 0 and {len(basis_vectors)-1}.")

# --- build the grid (same as you had) ---
gridmax = distance + 3.0
x = np.linspace(-gridmax, gridmax, 200)
y = np.linspace(-gridmax, gridmax, 200)
X, Y = np.meshgrid(x, y)
Z = np.zeros_like(X)
rvec = np.stack([X, Y, Z], axis=-1)

# --- pick what to visualize (same logic) ---
if type == "eigenvector":
    psi = sum(c * basis(rvec=rvec) for c, basis in zip(C[:, index], basis_vectors))
else:
    psi = basis_vectors[index](rvec=rvec)

# --- choose field & color normalization ---
if show == "wavefunction":
    field = np.real(psi)
    vmax = float(np.nanmax(np.abs(field))) + 1e-12
    norm = mpl.colors.TwoSlopeNorm(vmin=-vmax, vcenter=0.0, vmax=vmax)
    cmap = mpl.colormaps.get_cmap("bwr")   # <--- new API (no deprecation)
    cbar_label = r"$\Re\,\psi$"
else:  # "density"
    field = (psi * np.conj(psi)).real
    vmin, vmax = 0.0, float(np.nanpercentile(field, 99))
    norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)
    cmap = mpl.colors.LinearSegmentedColormap.from_list("wb", ["#ffffff", "#08306b"])
    cbar_label = r"$|\psi|^2$"

# --- take a 1D cut along y=0 (internuclear axis); average a thin band for smoothness ---
iy0 = int(np.argmin(np.abs(y - 0.0)))   # row closest to y=0
band = 1                                # average over rows [iy0-1 .. iy0+1]
iy_lo = max(0, iy0 - band)
iy_hi = min(field.shape[0], iy0 + band + 1)
line_x = x
line_y = np.nanmean(field[iy_lo:iy_hi, :], axis=0)

# --- make two stacked axes sharing x ---
fig = plt.figure(figsize=(6.6, 6.8))
gs = fig.add_gridspec(2, 2, height_ratios=[1,5], width_ratios=[40,2],
                      hspace=0.06, wspace=0.06)
ax_line = fig.add_subplot(gs[0, 0])
ax_img  = fig.add_subplot(gs[1, 0], sharex=ax_line)
cax     = fig.add_subplot(gs[:, 1])  # spans both rows

# top: line plot along x (through the atoms)
ax_line.plot(line_x, line_y, lw=1.8)
# vertical markers for nuclei
for xN in (RA[0], RB[0]):
    ax_line.axvline(xN, color="k", ls="--", lw=0.8, alpha=0.7)
ax_line.set_ylabel(cbar_label)

# choose sensible y-limits
if show == "wavefunction":
    L = np.nanmax(np.abs(line_y))
    ax_line.set_ylim(-1.05*L if L > 0 else -1, 1.05*L if L > 0 else 1)
    ax_line.axhline(0.0, color="k", lw=0.7, ls=":")
else:
    L = np.nanmax(line_y)
    ax_line.set_ylim(0.0, 1.05*L if L > 0 else 1.0)
ax_line.tick_params(axis="x", which="both",
                    bottom=False, top=False, labelbottom=False)

# bottom: contour image
cn = ax_img.contourf(X, Y, field, levels=80, cmap=cmap, norm=norm)
fig.colorbar(cn, cax=cax, label=cbar_label)

# nuclei markers on the image
ax_img.scatter([RA[0], RB[0]], [RA[1], RB[1]],
               s=60, c='k', edgecolors='w', linewidths=0.8, zorder=5)

#ax_img.set_aspect("scaled", adjustable="box")
ax_img.set_xlabel("x (bohr)")
ax_img.set_ylabel("y (bohr)")

# title
if type == "eigenvector":
    ax_line.set_title(f"Orbital {index}  (E = {E[index]:.4f} Ha)  — line cut (top) and z=0 slice (bottom)")
else:
    m = basis_vectors[index].keywords.get("m")
    l = basis_vectors[index].keywords.get("l")
    n = basis_vectors[index].keywords.get("n")
    ax_line.set_title(f"AO n={n}, l={l}, m={m} — line cut (top) and z=0 slice (bottom)")

plt.show()

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

def _annotate_matrix(ax, data, fmt="{:.2f}", max_n=20):
    n, m = data.shape
    if n*m > max_n*max_n:
        return
    A = np.abs(data); Amax = A.max() + 1e-12
    for (i, j), v in np.ndenumerate(data):
        ax.text(j, i, fmt.format(v),
                ha="center", va="center",
                color=("white" if A[i, j] > 0.6*Amax else "black"),
                fontsize=8)

def plot_energy_levels_on_ax(ax, energies, tol=2e-4, zero_cut=1e-6, title=None):
    E = np.asarray(energies, float)
    E = np.sort(E[np.abs(E) > float(zero_cut)])
    if E.size == 0:
        raise ValueError("No energies remain after zero_cut filtering.")
    Etot = E + 1/distance  # shift by +1/R to compare with H atom levels
    groups, cur = [], [Etot[0]]
    for e in Etot[1:]:
        if abs(e-cur[-1]) <= tol: cur.append(e)
        else: groups.append(cur); cur = [e]
    groups.append(cur)

    for i, grp in enumerate(groups):
        xs = i + (np.linspace(-0.2, 0.2, len(grp)) if len(grp)>1 else np.array([0.0]))
        for x, val in zip(xs, grp):
            ax.hlines(val, x-0.28, x+0.28)
        if len(grp)>1:
            ax.text(i, max(grp)+0.02, f"{len(grp)}×", ha="center", va="bottom", fontsize=9)

    ax.set_xlim(-0.6, len(groups)-0.4)
    ax.set_xlabel("Group (near-degenerate levels)", fontsize=12)
    ax.set_ylabel("Energy (Ha)", fontsize=12)
    ax.axhline(0.0, ls="--", lw=0.8)
    if title: ax.set_title(title)
    ax.grid(True, axis="y", ls=":", lw=0.5)

    # --- overlay hydrogen atom levels (red dashed) ---
    xmin, xmax = ax.get_xlim()
    xr = xmax - xmin
    for n in range(1, nmax + 1):
        E_H = - (1**2) / (2.0 * n * n)
        ax.hlines(E_H, xmin, xmax, colors="red", linestyles="--", linewidth=1.2)
        ax.text(xmax - 0.02*xr, E_H+0.02, f"H atom n={n}", color="red",
                va="center", ha="right", fontsize=9)

    return groups

def plot_h2p_summary_vertical_outside(
    S, H, energies, deg_tol=2e-4, zero_cut=1e-6,
    annotate=True, fmt="{:.2f}",
    figsize=(12, 6.5),
    width_ratios=(8.0, 0.8, 3.0),   # [left matrices, cbar column, right ladder] → right is thinner
):
    """
    Layout (2 rows × 3 cols):
      col0: Overlap (top) / Hamiltonian (bottom)
      col1: colorbar for Overlap (top) / colorbar for Hamiltonian (bottom)
      col2: Energy-level ladder (spans both rows, thinner panel)
    """
    Sreal = np.asarray(S.real)
    Hreal = np.asarray(H.real)

    cmap_div = mpl.colormaps.get_cmap("seismic")
    normS = mpl.colors.TwoSlopeNorm(vmin=-1.0, vcenter=0.0, vmax=1.0)
    Habs  = float(np.max(np.abs(Hreal)))
    normH = mpl.colors.TwoSlopeNorm(vmin=-Habs, vcenter=0.0, vmax=Habs)

    fig = plt.figure(figsize=figsize, layout="constrained")
    gs  = fig.add_gridspec(2, 3, height_ratios=[1, 1], width_ratios=width_ratios)

    # left column (stacked matrices)
    axS  = fig.add_subplot(gs[0, 0])
    axH  = fig.add_subplot(gs[1, 0])
    # slim colorbar column (separate axes per row)
    caxS = fig.add_subplot(gs[0, 1])
    caxH = fig.add_subplot(gs[1, 1])
    # right column (ladder, spans both rows)
    axE  = fig.add_subplot(gs[:, 2])

    # --- Overlap ---
    imS = axS.imshow(Sreal, cmap=cmap_div, norm=normS, aspect="auto")
    axS.set_title("Overlap Matrix (real part)")
    axS.set_xlabel("j"); axS.set_ylabel("i")
    cbS = fig.colorbar(imS, cax=caxS)
    cbS.set_ticks(np.linspace(-1, 1, 9))
    if annotate: _annotate_matrix(axS, Sreal, fmt=fmt)

    # --- Hamiltonian ---
    imH = axH.imshow(Hreal, cmap=cmap_div, norm=normH, aspect="auto")
    axH.set_title("Hamiltonian Matrix (real part)")
    axH.set_xlabel("j"); axH.set_ylabel("i")
    cbH = fig.colorbar(imH, cax=caxH)
    cbH.set_ticks(np.linspace(-Habs, Habs, 9))
    if annotate: _annotate_matrix(axH, Hreal, fmt=fmt)

    # --- Energy levels (right, thinner) ---
    plot_energy_levels_on_ax(axE, energies, tol=deg_tol, zero_cut=zero_cut,
                             title=f"Energy levels (|ΔE| ≤ {deg_tol:g} Ha)")

    return fig, (axS, axH, axE)

# ---- usage ----
# Make the right panel even thinner by reducing width_ratios[2], e.g. (8.0, 0.8, 2.2)
fig, (axS, axH, axE) = plot_h2p_summary_vertical_outside(S, H, E, width_ratios=(8.0, 0.5, 4))
plt.show()


In [ ]:
# @title PES Scan Parameters  { display-mode: "form" }
max_distance = 10.0 # @param {type:"number"}
min_distance = 1.0 # @param {type:"number"}
npoints = 10 # @param {type:"integer"}
nmax = 1  # @param {type:"integer"}
lmax = 0  # @param {type:"integer"}
integration_accuracy = "medium"  # @param ["coarse", "medium", "fine", "very fine"]

In [ ]:
# @title PES Scan Simulation and Visualization { display-mode: "form" }
ns = []
orbitals = []
for i in range(1,nmax+1):
    for j in range(0, min(lmax+1,i)):
        for m in range(-j, j+1):
            ns.append(i)
            orbitals.append(partial(hydrogen_orbital, n=i, l=j, m=m))

if integration_accuracy == "coarse":
    n_rad, n_th, n_ph = 10, 10, 12
elif integration_accuracy == "medium":
    n_rad, n_th, n_ph = 20, 20, 24
elif integration_accuracy == "fine":
    n_rad, n_th, n_ph = 40, 30, 24
elif integration_accuracy == "very fine":
    n_rad, n_th, n_ph = 50, 40, 36

distances = np.linspace(min_distance, max_distance, npoints)
energies = []
for distance in distances:
    RA = np.array([-distance/2, 0.0, 0.0])  # nucleus A at -R/2 on x-axis
    RB = np.array([ distance/2, 0.0, 0.0])  # nucleus B at +R/2 on x-axis
    basis_vectors = []
    basis_energies = []
    for n, orbital in zip(ns, orbitals):
        for atom_position in [RA, RB]:
            basis_vectors.append(partial(orbital, r0=atom_position))
            basis_energies.append(hydrogen_energy(n))
    H, S = hamiltonian_matrix(basis_vectors, basis_energies, RA, RB, n_rad, n_th, n_ph, alpha=1.0)
    E, C = generalized_eigh_lowdin(H, S, rel_cut=5e-2)
    energies.append(E+1/distance)
energies = np.array(energies)

# @title Visualize the PES Scan { display-mode: "form" }
import plotly.graph_objects as go
from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=1)
for i in range(energies.shape[1]):
    fig.add_trace(go.Scatter(x=distances, y=energies[:,i], mode='lines+markers', name=f'State {i}'), row=1, col=1)
fig.update_layout(title='H2+ Potential Energy Surface Scan', xaxis_title='Distance (bohr)', yaxis_title='Energy (Ha)')
fig.show()